**<주제>**  

**기존고객** : 구매데이터에서 소분류명(혹은 소분류 코드?)/고객ID/구매횟수(평가기준)를 기반으로 아이템기반 협업 필터링 데이터셋을 구성한다.  
→  구매 상품 이력을 기반으로 한 개인 맞춤 상품 추천 시스템 구현  
  
 ------------   
  
**신규고객** : 구매데이터에서 사용자 기반 협업 필터링으로 데이터셋을 구성하기 위해 필요한 고객 특성들을 칼럼에 반영한다.   
→  신규고객은 구매이력이 없기 때문에, 고객 특성(연령/성별 등)을 반영한 맞춤 추천 시스템 구현

In [1]:
!apt install libnvrtc8.0
!pip uninstall -y mxnet-cu80 && pip install mxnet-cu80==1.1.0
!pip install turicreate

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package libnvrtc8.0
E: Couldn't find any package by glob 'libnvrtc8.0'
E: Couldn't find any package by regex 'libnvrtc8.0'
     |████████████████████████████████| 274.9 MB 2.6 kB/s 
     |████████████████████████████████| 88 kB 7.4 MB/s 
     |████████████████████████████████| 5.0 MB 78.1 MB/s 
     |████████████████████████████████| 56 kB 4.1 MB/s 
     |████████████████████████████████| 132 kB 74.6 MB/s 
  ERROR: Failed building wheel for numpy
  Running setup.py clean for numpy
  ERROR: Failed cleaning build dir for numpy
Failed to build numpy
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: idna
    Found existing installation: idna 2.10
    Uninstalling idna-2.10:
      Successfully uninstalled idna-2.10
  Attempting uninstall: request

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

In [3]:
# 나눔 폰트 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cashe -fv
!rm ~/.cache/matplotlib -rf
# 순서 : 마운트초기화 > 폰트설치 > 런타임다시시작 > 구글드라이브 마운트 > 나머지 코드...

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (6,436 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 155013 files and d

In [4]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
my_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', my_path)
sys.path.insert(0,my_path)

Mounted at /content/drive


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!pip3 install pickle5

     |████████████████████████████████| 132 kB 5.4 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219309 sha256=9a094b4c515585d3b262bf8ae2e0efa84afc55cab4f33a1a507af43a7206f9ef
  Stored in directory: /root/.cache/pip/wheels/7e/6a/00/67136a90d6aca437d806d1d3cedf98106e840c97a3e5188198
Successfully built pickle5


In [7]:
# 피클파일 불러오기
import pickle
with open("/content/drive/MyDrive/cakd3_1차프로젝트_2조/dataset/Ldata0920.pkl", "rb") as fh: Ldata = pickle.load(fh)

FileNotFoundError: ignored

In [ ]:
# 피클파일 불러오기2 
import pickle5 as pickle 
with open("/content/drive/MyDrive/cakd3_1차프로젝트_2조/dataset/draft.pkl", "rb") as fh: data = pickle.load(fh)

In [ ]:
data.to_pickle(path_to_protocol4)
data.head()

In [ ]:
data1 = Ldata['구매상품']

In [ ]:
!python --version

In [ ]:
!apt install libnvrtc8.0
!pip uninstall -y mxnet-cu80 && pip install mxnet-cu80==1.1.0
!pip install turicreate

In [ ]:
# %load_ext autoreload
# %autoreload 2

import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import train_test_split

import sys
# sys.path.append("..")

In [ ]:
# 사용자 데이터셋
data = data1.groupby(['고객번호','소분류명'])['소분류명'].count()

In [ ]:
data = data.to_frame()
data.rename(columns={'소분류명':'purchase_count'}, inplace=True)
data.reset_index(inplace=True)

In [ ]:
data.rename(columns={'고객번호':'customerId', '소분류명':'productId'}, inplace=True)
data.head()

In [ ]:
# 더미데이터
def create_data_dummy(data): 
  data_dummy = data.copy() 
  data_dummy['purchase_dummy'] = 1 
  return data_dummy 
data_dummy = create_data_dummy(data) 

In [ ]:
# 항목 값 정규화
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix.head()

In [ ]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
df_matrix_norm.head()

In [ ]:
d = df_matrix_norm.reset_index() 
d.head()

In [ ]:
d.index.names = ['scaled_purchase_freq'] 

In [ ]:
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
data_norm.tail(100)

In [ ]:
import turicreate as tc

In [ ]:
# 결합
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

In [ ]:
# train_test_split
def split_data(data):
    '''
    Splits dataset into training and test set.
    Args:
        data (pandas.DataFrame)     
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data


train_data, test_data = split_data(data) 
train_data_dummy, test_data_dummy = split_data(data_dummy) 
train_data_norm, test_data_norm = split_data(data_norm) 

In [ ]:
# 변수정의
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(data[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to display the first few rows in an output dataset

In [ ]:
# 모델 기능 정의
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

## 협업 필터링 모델
- 인덱스 값이 고유한 고객 아이디를 나타내고 열값이 고유한 제품 아이디를 나타내는 사용자 항목 행렬 제작
- 항목 간 유사성 행렬 제작
  - 제품간 유사도 계산
- 구매 예측

In [ ]:
# 구매 카운트 기준
name = 'popularity' 
target = 'purchase_count' 
popularity = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

# 구매 더미 기준
name = 'popularity' 
target = 'purchase_dummy' 
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

# 스케일링 된 구매 카운트 기준
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

In [ ]:
# (7.2)협업 필터링 모델 코싸인 유사도

# 구매 카운트 기준
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

# 구매 더미 기준
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

# 스케일링 된 구매 카운트 기준
name = 'cosine' 
target = 'scaled_purchase_freq' 
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

In [ ]:
# (7.3)협업 필터링 모델 고객간 피어슨 유사도 

# 구매 카운트 기준
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

# 구매 더미 기준
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

# 스케일링 된 구매 카운트 기준
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

In [ ]:
# 모델 평가 - RMSE, recall, precision
models_w_counts = [popularity, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]

names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

In [ ]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

In [ ]:
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)

In [ ]:
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

In [ ]:
# 평가 결과 출력
final_model = tc.item_similarity_recommender.create(tc.SFrame(data_norm), user_id=user_id, item_id=item_id,
                                                    target='purchase_dummy', similarity_type='cosine')
recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

In [ ]:
# csv로 출력
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()
 
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customerId').set_index('customerId')
    if print_csv:
        df_output.to_csv('../output/option1_recommendation.csv')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

df_output = create_output(pear_norm, users_to_recommend, n_rec, print_csv=True)
print(df_output.shape)
df_output.head()

In [ ]:
# 고객 추천 기능
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]